Este documento fornece uma implementação de algoritmos de aprendizagem curricular dentro da implementação de PINNs da biblioteca DeepXDE. Apresentamos dois algoritmos distintos: um algoritmo manual, onde uma lista de amplitudes é fornecida pelo usuário; e um método adaptativo, onde tal sequência é construída a partir de regras preestabelecidas.
O código foi estruturado em diversas seções – as únicas importantes para o entendimento do funcionamento do algoritmo são a "Implementação de PINNs para solução da equação de KdV no DeepXDE", "Funções para cálculo de perda e critério de aproximação do modelo" e "Curriculum Learning".

### **Implementação de PINNs para solução da equação de KdV no DeepXDE**

In [ ]:
#@title Instalando e importando bibliotecas
!pip install deepxde -q

import os
os.environ['DDE_BACKEND'] = 'pytorch'  # Existem outras opções, como tensorflow.compat.v1, pytorch, jax, paddle.

import deepxde as dde
import numpy as np
import tensorflow as tf
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from math import sqrt, floor
from datetime import datetime

# Função usada internamente no DeepXDE para alterar solução-referência da PDE;
# a solução é usada no treinamento dos pontos das condições inicial e de borda.
from deepxde.icbc.boundary_conditions import npfunc_range_autocache
from deepxde import utils

# Fixando semente. Necessário para garantir determinismo.
seed_value = 0
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
dde.config.set_random_seed(seed_value)

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.
2025-07-26 13:19:04.515368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-26 13:19:04.515450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-26 13:19:04.517385: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-26 13:19:04.527214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following inst

In [ ]:
# Intervalos espacial e temporal onde será resolvida a equação; com tempo de 0 até tmax
xmin, xmax = [-50, 50]
tmax = 60

amax = 3

# Criando um intervalo interespaçado com x_num pontos; Será usado no cálculo de erro e plot das soluções.
x_num = 400
x = np.linspace(xmin, xmax, x_num)

# Definindo a geometria do espaço onde será resolvida a equação;
# neste caso é um retângulo de uma dimensão espacial e uma temporal.
geom = dde.geometry.Interval(xmin, xmax)
timeDomain = dde.geometry.TimeDomain(0,tmax)
geomtime = dde.geometry.GeometryXTime(geom, timeDomain)

In [ ]:
# Implementando o operador diferencial da equação de Korteweg-de-Vries.
def PDE(x, y):
  # x = (x, t)
  # y = u(x, t)

  dy_t = dde.grad.jacobian(y, x, i=0, j=1)
  dy_x = dde.grad.jacobian(y, x, i=0, j=0)
  dy_xxx = dde.grad.hessian(dy_x, x, i=0, j=0)

  PDE = dy_t - (3/2)*y*dy_x - dy_xxx/6
  return PDE

In [ ]:
# Definindo a solução exata da equação acima
def pde_sol(X, a=1):
  k = sqrt(3*a)/2
  c = -a/2

  E = k*(X[:, 0:1] - c*X[:, 1:2])
  return a*np.power(np.cosh(E), -2)

In [ ]:
"""
Condições inicial e de borda.
Este trecho mostra que os pontos na codição inicial e na borda da geometria geomtime
serão calculados a partir de nossa solução pde_sol.
"""
bc = dde.icbc.DirichletBC(geomtime, pde_sol, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, pde_sol, lambda _, on_initial: on_initial)

In [ ]:
"""
Dados para o treinamento do modelo, incluem: a geometria; a equação;
as condições inicial e de borda e o número de pontos no interior do domínio,
na borda e na condição inicial.
"""

data = dde.data.TimePDE(
                        geomtime,
                        PDE,
                        [bc, ic],
                        num_domain=30000,
                        num_boundary=300,
                        num_initial=300
                        )

In [ ]:
"""
Definição da rede como do tipo Feedforward, a arquitetura é definida como
[Número de Inputs] + [Neurônios por camada] * [Número de camadas] + [Número de Outputs].
A função de ativação escolhida é tanh e a inicialização dos pesos é Glorot normal.
"""
net = dde.nn.FNN([2] + [6] * 7 + [1], "tanh", "Glorot normal")

In [ ]:
"""
Aqui criamos um objeto modelo que toma os dados de treinamento e a arquitetura da rede.
Em seguida compilamos o modelo com o otimizador adam e a taxa de aprendizado escolhida.
"""
model = dde.Model(data, net)
lr = 1e-4
model.compile("adam", lr=lr)

Compiling model...
'compile' took 0.515943 s



In [ ]:
"""
Alteramos as condições de borda e inicial com a função previamente importada.
Por algum motivo o treinamento isolado (utilizando apenas model.train) não está
funcionando sem esse trecho; de qualquer modo é inofensivo.
"""

func = lambda X: pde_sol(X, a=1)
model.data.bcs[0].func = npfunc_range_autocache(utils.return_tensor(func))
model.data.bcs[1].func = npfunc_range_autocache(utils.return_tensor(func))

## **Utils**
Definindo funcionalidades de cálculo de erro, gráficos e armazenamento de dados.

In [ ]:
#@title Funções de cálculo de erro relativo l2
"""
A função get_arr_X(t) cria uma matriz X duas colunas: x e t.
A primeira coluna é aquele vetor de pontos interespaçados definido anteriormente.
A segunda coluna é um vetor em que cada coordenada é a constante t.
Usaremos essa função para cálculo do erro e criação de gráficos.
"""
def get_arr_X(t):
  t_array = np.full_like(x, t)
  X = np.stack((x, t_array), axis=1)
  return X


# Calcula o erro relativo L2 entre a solução exata e a previsão do modelo
def error(a, t, model):
  X = get_arr_X(t)
  y_pred = model.predict(X)
  y_true = pde_sol(X, a)
  return dde.metrics.l2_relative_error(y_true, y_pred)

In [ ]:
#@title Funções para cálculo de perda e critério de aproximação do modelo.

# Essa função é usada para calcular a perda da rede em seu estado atual
def last_loss(model, monitor_loss_train=True):
    return model._outputs_losses(
        monitor_loss_train,
        model.train_state.X_train,
        model.train_state.y_train,
        model.train_state.train_aux_vars,
        )[1]


"""
Essa função é usada para definir se o modelo aproximou ou não segundo nossos critérios.
Por padrão dizemos que o modelo não aproxima na primeira iteração; essa convenção será útil na construção dos algoritmos.
O critério de aproximação é arbitrário – escolhemos minimizar a média aritmética dos erros, mas sinta-se livre para experimentar
"""
def modelo_aproximou(model, baseline, monitor_loss_train = True):
    if model.train_state.step == 0:
      return False

    elif sum(last_loss(model, monitor_loss_train))/3 >= baseline:
      return False

    else:
      return True

In [ ]:
#@title Armazenamento dos dados de treinamento

# Essa função é chamada durante o treinamento para salvar os dados na lista model.train_state.train_data
# aqui é salvo: amplitude, iteração, número de iterações de cada etapa e tabelas das perdas de cada etapa.
def save_data(model, amplitude, display_every):
  step = model.train_state.step
  step_diff = step - model.train_state.steps_before_train

  lastlosstrain = last_loss(model, True)
  lastlosstest = last_loss(model, False)

  D = floor(step_diff/display_every)
  if D == 0:
    loss_test = model.losshistory.loss_test[-1]
    loss_train = model.losshistory.loss_train[-1]
  else:
    loss_test = model.losshistory.loss_test[-D:-1]
    loss_train = model.losshistory.loss_train[-D:-1]

  model.train_state.train_data.append([amplitude, step, step_diff, lastlosstrain, lastlosstest, loss_train, loss_test])

# Essa função é usada para salvar, durante a execução dos algoritmos, as perdas em cada iteração –
# seja uma iteração dentro do treinamento ou uma iteração do loop (caso o modelo aproxime em 0 passos).
def save_loss_data(model):
    counter = model.train_state.counter
    loss = last_loss(model)
    model.train_state.loss_data[0].append(counter)
    model.train_state.loss_data[1].append(loss)


# Salva os dados da lista model.train_state.train_data num arquivo .txt
def write_data(model, filepath, monitor_loss_train=True):
  with open(filepath, 'w') as f:
    for batch in model.train_state.train_data:
      f.write(
          f"""
          AMPLITUDE: {batch[0]}\t
          STEP: {batch[1]}\t
          STEP_DIFF: {batch[2]}\t
          LAST_LOSS_TRAIN: {batch[3]}\t
          LAST_LOSS_TEST: {batch[4]}\t
          \n\n"""
          )

      if monitor_loss_train:
        f.write(f"LOSS_TRAIN:\n")
        for loss in batch[5]:
          f.write(f"{loss}\n")
      else:
        f.write(f"LOSS_TEST:\n")
        for loss in batch[6]:
          f.write(f"{loss}\n")

      f.write("\n\n\n")

In [ ]:
#@title Gráficos/Gif

# plot do erro l2 para amplitude fixa e tempo variando
def error_plot(model, filepath, amplitude):
    fig, ax = plt.subplots()
    T = np.linspace(0, tmax, 100)
    err = lambda a, t : error(a, t, model=model)
    v_error = np.vectorize(err)
    ax.plot(T, v_error(amplitude, T))
    ax.set_xlabel("t")
    ax.set_ylabel("error")
    plt.savefig(f"{filepath}/l2_error_vs_t_a={amplitude:.7}.png")
    plt.close()


# plot das perdas do modelo
# o campo avg=True pega a média aritmética das perdas
# o campo steps pega os valores máximo e mínimo em intervalos de steps_iterações
# e constrói três gráficos: um limita superiormente, outro inferiormente e o último toma a média aritmética entre o mínimo e máximo.
def loss_plot(model, filepath, avg = True, steps = None):
    fig, ax = plt.subplots()
    c, l = model.train_state.loss_data
    if avg:
        avg_l = [sum(i)/3 for i in l]
        if steps is None:
            ax.plot(c, avg_l)
        else:
            max_l = []
            min_l = []
            max_c = []
            min_c = []
            n = floor(len(c)/steps)
            for i in range(n):
                if i == n-1:
                    avg_slice = avg_l[i*steps : -1]
                    c_slice = c[i*steps : -1]
                else:
                    avg_slice = avg_l[i*steps : (i+1)*steps]
                    c_slice = c[i*steps : (i+1)*steps]


                min_l.append(min(avg_slice))
                max_l.append(max(avg_slice))
                flag_min = True
                flag_max = True
                for k, l in enumerate(c_slice):
                    if avg_slice[k] == min(avg_slice) and flag_min:
                        min_c.append(l)
                        flag_min = False

                    if avg_slice[k] == max(avg_slice) and flag_max:
                        max_c.append(l)
                        flag_max = False

                    if flag_min is False and flag_max is False:
                        break

            avg_c = [(min_c[j] + max_c[j])/2 for j in range(len(min_c))]
            midpoints = [(min_l[j] + max_l[j])/2 for j in range(len(min_l))]

            ax.fill_between(avg_c, max_l, min_l, alpha=0.5)
            ax.plot(avg_c, midpoints)

    else:
      ax.set_prop_cycle(color=['red', 'blue', 'green'])
      ax.plot(c, l, alpha=0.7)

    ax.set_xlabel("meta_iteração")
    ax.set_ylabel("loss")
    ax.set_yscale("log")
    if steps is None:
        plt.savefig(f"{filepath}/loss_avg={avg}.png")
    else:
        plt.savefig(f"{filepath}/loss_avg={avg}; steps={steps}.png")
    plt.close()

# Dada uma amplitude, cria o gif comparando as soluções
def gif(model, filepath, a):
    Figure = plt.figure()
    predicted_line = plt.plot([], 'b-')[0]
    solution_line = plt.plot([], 'r--')[0]
    plt.xlim(xmin, xmax)
    plt.ylim(-0.1*a, a*1.1)
    frames=120
    def AnimationFunction(frame):
        K= (tmax)/frames
        t = K*frame
        plt.title(f"t={t:.2f}")
        X = get_arr_X(t=t)
        y_pred = model.predict(X)
        y_sol = pde_sol(X, a)
        predicted_line.set_data((x, y_pred))
        solution_line.set_data((x, y_sol))

    anim_created = FuncAnimation(Figure, AnimationFunction, frames=frames, interval=25)
    writer = animation.PillowWriter(fps=24,
                                    metadata=dict(artist='Me'),
                                    bitrate=1800)
    anim_created.save(f'{filepath}/prediction_vs_exact_amplitude{a}.gif', writer=writer)
    plt.close()

## **Curriculum Learning**

In [ ]:
#@title Callback :  CurriculumLearning
"""
Os objetos da classe callback são passados para a função model.train, que
executa o treinamento do modelo. A sua função é executar código durante o treinamento.
Em especial tem-se as funções on_epoch_begin e on_epoch_end, que são executadas
antes e depois de cada iteração.
O objeto callback criado abaixo tem uma função simples, as únicas funções executadas
são on_epoch_begin e on_epoch_end.
Neste caso função on_epoch_begin só tem a funcionalidade de salvar dados.
A função on_epoch_end determina se o modelo aproximou ou não ao final de cada iteração,
caso aproximou o treinamento é interrompido.
No caso da não aproximação, o treinamento continua ou não segundo os seguintes critérios:
Se atingiu o número máximo de iterações o treinamento para e o algoritmo é interrompido.
Se o número de iterações for menor que uma porcentagem (variável ratio) do número de iterações
da primeiro etapa, o treinamento para. Neste caso o algoritmo é interrompido somente se for o Manual.
Os sinalizadores de interrupção do treinamento e interrupção do algoritmo são, respectivamente,
model.stop_training e model.train_state.end_loop.
"""



class CurriculumLearning(dde.callbacks.Callback):
  def __init__(
      self,
      amplitude,
      model,
      algoritmo_manual,    # True ou False; False caso adaptativo.
      baseline=1e-9,
      ratio=0.8,
      max_iterations=100000,
      monitor_loss_train = True,
      ):
    super().__init__()
    self.baseline = baseline
    self.max_iterations = max_iterations
    self.monitor_loss_train = monitor_loss_train
    self.amplitude = amplitude
    self.ratio = ratio
    self.model=model
    self.algoritmo_manual = algoritmo_manual


  def on_epoch_begin(self):
    if self.model.train_state.step > 0:
        save_loss_data(self.model)
        self.model.train_state.counter += 1
    else:
        pass

  def on_epoch_end(self):
    step = self.model.train_state.step
    step_diff = step - self.model.train_state.steps_before_train
    first_batch_steps = self.model.train_state.first_batch_steps

    aproximou = modelo_aproximou(self.model, self.baseline, self.monitor_loss_train)

    steps_le_max = step < self.max_iterations
    steps_le_first_batch = True

    if first_batch_steps is not None:
      steps_le_first_batch = step_diff < floor(self.ratio * first_batch_steps)



    if (not aproximou) and steps_le_first_batch and steps_le_max:
      return

    else:
      self.model.stop_training = True

      if aproximou:
        print(f"APROXIMOU EM {step_diff} PASSOS")
        if first_batch_steps is None:
          self.model.train_state.first_batch_steps = step

      elif not steps_le_first_batch:
        print("NÃO APROXIMOU")
        if self.algoritmo_manual:
          self.model.train_state.end_loop = True

      else:
        print("NÃO APROXIMOU. MÁXIMO DE ITERAÇÕES ALCANÇADO")
        self.model.train_state.end_loop = True


In [ ]:
#@title Algoritmos
class Train():
    def __init__(
        self,
        model,
        max_iterations,
        max_batch_iterations,
        weights_path=None,
        baseline=1e-9,
        ratio=0.1,
        monitor_loss_train = True,
        display_every=100,
        first_batch_steps=None,
        loss_plot_steps=100
    ):
        self.model = model
        self.max_iterations = max_iterations
        self.max_batch_iterations = max_batch_iterations
        self.baseline = baseline
        self.ratio = ratio
        self.monitor_loss_train = monitor_loss_train
        self.display_every = display_every
        self.model.train_state.first_batch_steps = first_batch_steps
        self.loss_plot_steps = loss_plot_steps

        self.model.train_state.end_loop = False
        self.model.train_state.savepath = None
        self.model.train_state.steps_before_train = 0

        self.model.train_state.train_data = list()
        self.model.train_state.loss_data = [[],[]]
        self.model.train_state.counter = 0
        self.pesos_carregados = False


        if weights_path is not None:
            self.model.restore(weights_path)
            self.model.predict([0,0])
            self.model.train_state.savepath = weights_path
            self.pesos_carregados = True



    # Cria pastas para salvar os dados de treinamento
    def cria_pastas(self):
        dia = datetime.now()
        self.dia = f"{dia.day:02}{dia.month:02}{dia.year}_{dia.hour:02}{dia.minute:02}"
        !mkdir modelos
        !mkdir modelos/{self.dia}
        !mkdir modelos/{self.dia}/weights
        !mkdir modelos/{self.dia}/graficos


    def salva_dados(self, a):
        write_data(self.model, f"modelos/{self.dia}/data_loss_train.txt")
        write_data(self.model, f"modelos/{self.dia}/data_loss_test.txt", False)

        loss_plot(self.model, f"modelos/{self.dia}/graficos")
        loss_plot(self.model, f"modelos/{self.dia}/graficos", avg=True, steps=self.loss_plot_steps)
        gif(self.model, f"modelos/{self.dia}/graficos", float(a))
        error_plot(self.model, f"modelos/{self.dia}/graficos", float(a))


    def algoritmo_manual(
        self,
        amplitude_range,
    ):
        """
        O algoritmo manual itera sobre uma lista de amplitudes amplitude_range fornecida pelo usuário.
        No início de cada iteração altera as condições inicial e de bordo do modelo para condizer com
        a amplitude atual. É realizado um treinamento com a função de custo da amplitude atual.
        O algoritmo é interrompido quando em alguma etapa o treinamento excede uma porcentagem do
        número de iterações realizado na primeira amplitude – a porcentagem é determinada pela variável
        ratio, do objeto callback (instância da classe CurriculumLearning).
        Como o objeto model é o mesmo durante cada iteração os pesos são conservados entre o final do
        treinamento numa amplitude e início do treinamento na próxima.
        """

        self.cria_pastas()

        for a in amplitude_range:
            if self.model.train_state.end_loop:
                break
            print(f"a = {a}")

            # Altera condições inicial e de bordo para a solução na amplitude atual.
            func = lambda X: pde_sol(X, a=a)
            self.model.data.bcs[0].func = npfunc_range_autocache(utils.return_tensor(func))
            self.model.data.bcs[1].func = npfunc_range_autocache(utils.return_tensor(func))

            self.model.train_state.steps_before_train = self.model.train_state.step
            if modelo_aproximou(self.model, self.baseline, self.monitor_loss_train):
                save_data(self.model, a, self.display_every)
                print(f"APROXIMOU EM 0 PASSOS")
                continue

            else:
                callback = CurriculumLearning(
                    a,
                    self.model,
                    algoritmo_manual = True,
                    ratio = self.ratio,
                    baseline = self.baseline,
                    max_iterations = self.max_iterations,
                    monitor_loss_train = self.monitor_loss_train
                )

                self.model.train(
                    iterations = self.max_batch_iterations,
                    display_every = self.display_every,
                    callbacks = [callback]
                )

                if modelo_aproximou(self.model, self.baseline, self.monitor_loss_train):
                    self.model.save(f"modelos/{self.dia}/weights/aproximou_em_a={a}__iteracao")
                    save_data(self.model, a, self.display_every)

        self.model.save(f"modelos/{self.dia}/weights/maxiters_em_a={a}__iteracao")
        self.salva_dados(amplitude_range[-1])


    def incrementa_amplitude(self, a, a_max, step_size):
        if a + step_size > a_max:
            return a_max
        else:
            return a + step_size


    def algoritmo_adaptativo(
        self,
        a_min,
        a_max,
        step_size=2**(-4),
        r=2**(-1),
    ):
        """
        O algoritmo adaptativo toma como entrada uma amplitude inicial a_min e uma amplitude final a_max,
        além de um step_size, indicando o tamanho do passo inicial, variáveis fornecidas pelo usuário.
        No início de cada iteração o algoritmo verifica se houve convergência imediata, caso positivo
        o tamanho do passo é dobrado e incrementa-se à amplitude o novo tamanho de passo, então o
        algoritmo prossegue para a próxima iteração.
        Caso não haja convergência imediata o algoritmo realiza uma etapa de treinamento na amplitude
        atual, se há convergência o modelo salva os pesos numa pasta para possível uso posterior.
        Se a convergência foi "rápida" – quando o número de iterações é menor que uma porcentagem (fixada 1%)
        das iterações da primeira etapa, o algoritmo dobra o tamanho do passo; em todo caso a amplitude é
        incrementada e se prossegue para a próxima iteração.
        Caso não haja convergência após o treinamento, o tamanho de passo é reduzido para uma porcentagem
        do tamanho atual (variável r), os pesos da última amplitude bem-sucedida são carregados e
        define-se a próxima amplitude como a soma da última amplitude bem-sucedida com o novo passo reduzido.
        Assim o algoritmo prossegue até que haja convergência na amplitude final ou que o número máximo de
        iterações seja atingido.
        """
        if a_min > a_max:
            raise Exception("Amplitude inicial deve ser menor que a final.")
        self.cria_pastas()
        a = a_min
        while a <= a_max:
            if self.model.train_state.end_loop:
                break

            print(f"a = {a}")
            print(f"step_size = {step_size}")
            func = lambda X: pde_sol(X, a=a)
            self.model.data.bcs[0].func = npfunc_range_autocache(utils.return_tensor(func))
            self.model.data.bcs[1].func = npfunc_range_autocache(utils.return_tensor(func))

            self.model.train_state.steps_before_train = self.model.train_state.step
            steps_before_train = self.model.train_state.steps_before_train

            if modelo_aproximou(self.model, self.baseline, self.monitor_loss_train):
                save_loss_data(self.model)
                save_data(self.model, a, self.display_every)
                if a == a_max:
                    break

                step_size = 2 * step_size
                a = self.incrementa_amplitude(a, a_max, step_size)
                self.model.train_state.counter += 1
                continue

            else:
                callback = CurriculumLearning(
                    a,
                    self.model,
                    algoritmo_manual = False,
                    ratio = self.ratio,
                    baseline = self.baseline,
                    max_iterations = self.max_iterations,
                    monitor_loss_train = self.monitor_loss_train
                )

                self.model.train(
                    iterations = self.max_batch_iterations,
                    display_every = self.display_every,
                    callbacks = [callback]
                )
                save_data(self.model, a, self.display_every)


                if modelo_aproximou(self.model, self.baseline, self.monitor_loss_train):
                    self.model.save(f"modelos/{self.dia}/weights/aproximou_em_a={a}__iteracao")

                    step = self.model.train_state.step
                    step_diff = step - steps_before_train

                    self.model.train_state.savepath = f"modelos/{self.dia}/weights/aproximou_em_a={a}__iteracao-{step}.pt" # MUDAR A DEPENDER DO BACKEND
                    if a == a_max:
                        break
                    aproximou_rapido = self.model.train_state.first_batch_steps is not None and (step_diff <= 0.01 * self.model.train_state.first_batch_steps)
                    if aproximou_rapido:
                        step_size = 2 * step_size
                    a = self.incrementa_amplitude(a, a_max, step_size)

                else:
                    if a == a_min and not self.pesos_carregados:
                        raise Exception("O método espera convergência na primeira amplitude.")

                    if self.model.train_state.step == self.max_iterations:
                        self.model.save(f"modelos/{self.dia}/weights/max_iters_em_a={a}")
                        break
                    a -= step_size
                    step_size = r * step_size
                    a = self.incrementa_amplitude(a, a_max, step_size)
                    self.model.restore(self.model.train_state.savepath)
                    self.model.predict([0,0])

        self.salva_dados(a_max)


# Testes

In [ ]:

train = Train(
    model,
    max_iterations = 38048,
    max_batch_iterations = 38048,
    first_batch_steps = 62241,
    ratio = 0.02,
    weights_path = f"onde/estao_os/pesos.pt" # remover linha ou deixar None se não for carregar pesos
)


/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
step_size = 2**(-10)
r = 2**(-5)
a_min = 1 + step_size
train.algoritmo_adaptativo(a_min, 1.104, step_size, r)

mkdir: cannot create directory ‘modelos’: File exists
a = 1.0009765625
step_size = 0.0009765625
Training model...

Step      Train loss                        Test loss                         Test metric
0         [2.56e-09, 9.24e-12, 1.16e-08]    [2.56e-09, 9.24e-12, 1.16e-08]    []  
100       [2.97e-09, 3.54e-09, 4.31e-09]    [2.97e-09, 3.54e-09, 4.31e-09]    []  
200       [5.76e-09, 1.98e-09, 2.94e-09]    [5.76e-09, 1.98e-09, 2.94e-09]    []  
300       [2.59e-09, 3.24e-12, 4.61e-10]    [2.59e-09, 3.24e-12, 4.61e-10]    []  
400       [2.88e-09, 4.70e-10, 7.58e-10]    [2.88e-09, 4.70e-10, 7.58e-10]    []  
500       [2.58e-09, 7.91e-10, 1.38e-09]    [2.58e-09, 7.91e-10, 1.38e-09]    []  
600       [3.04e-09, 1.54e-10, 5.72e-10]    [3.04e-09, 1.54e-10, 5.72e-10]    []  
700       [2.56e-09, 9.08e-11, 5.75e-10]    [2.56e-09, 9.08e-11, 5.75e-10]    []  
800       [5.28e-09, 8.05e-10, 1.23e-09]    [5.28e-09, 8.05e-10, 1.23e-09]    []  
APROXIMOU EM 884 PASSOS

Best model at step 300: